In [1]:
from sklearn.model_selection import train_test_split
import os
import shutil

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [6]:
def generate_train_test_split():
    art_styles = os.listdir('data_resized')  # 20 styles
    for style in art_styles:
        image_paths = [os.path.join('data_resized', style, img) for img in os.listdir(os.path.join('data_resized', style))]
        train_paths, test_paths = train_test_split(image_paths, test_size=0.2)
        train_paths, val_paths = train_test_split(train_paths, test_size=0.2)
        
        # Create directories for training, validation, and test sets
        os.makedirs('data_train/{}/'.format(style), exist_ok=True)
        os.makedirs('data_val/{}/'.format(style), exist_ok=True)
        os.makedirs('data_test/{}/'.format(style), exist_ok=True)

        # Move images to the corresponding directories
        for path in train_paths:
            shutil.copy(path, f'data_train/{style}/{os.path.basename(path)}')
        for path in val_paths:
            shutil.copy(path, f'data_val/{style}/{os.path.basename(path)}')
        for path in test_paths:
            shutil.move(path, f'data_test/{style}/{os.path.basename(path)}')

In [7]:
generate_train_test_split()